[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jkitchin/s26-06642/blob/main/dsmles/03-intermediate-pandas/intermediate-pandas.ipynb)

# Module 03: Intermediate Pandas

Advanced data manipulation techniques for real-world data analysis.

## Learning Objectives

1. Use groupby for split-apply-combine operations
2. Merge and join DataFrames
3. Reshape data with pivot and melt
4. Apply custom functions efficiently
5. Work with datetime data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create sample experimental data
np.random.seed(42)

experiments = pd.DataFrame({
    'experiment_id': [f'EXP{i:03d}' for i in range(1, 51)],
    'date': pd.date_range('2024-01-01', periods=50, freq='D'),
    'catalyst': np.random.choice(['Pt/Al2O3', 'Pd/Al2O3', 'Ru/Al2O3'], 50),
    'temperature': np.random.choice([300, 350, 400, 450, 500], 50),
    'pressure': np.random.uniform(1, 5, 50).round(1),
    'conversion': np.random.uniform(0.3, 0.95, 50).round(3),
    'selectivity': np.random.uniform(0.7, 0.98, 50).round(3)
})

experiments['yield'] = (experiments['conversion'] * experiments['selectivity'] * 100).round(1)
experiments.head(10)

## GroupBy: The Split-Apply-Combine Pattern

GroupBy is arguably the most powerful Pandas operation. It lets you answer questions like:
- "What's the average yield for each catalyst?"
- "What's the best-performing temperature for each reactor?"
- "How does conversion vary by day of week?"

### The Mental Model: Split-Apply-Combine

1. **Split**: Divide data into groups based on one or more columns
2. **Apply**: Perform some operation on each group
3. **Combine**: Combine results back into a DataFrame

This is the Pandas equivalent of SQL's `GROUP BY`, but more flexible.

### When to Use GroupBy

- Comparing performance across categories (catalysts, reactors, operators)
- Aggregating replicate experiments
- Computing statistics within subgroups
- Normalizing data within groups

In [ ]:
# Basic groupby: mean by catalyst
experiments.groupby('catalyst')['yield'].mean()

In [ ]:
# Multiple aggregations
experiments.groupby('catalyst')['yield'].agg(['mean', 'std', 'min', 'max', 'count'])

In [ ]:
# Group by multiple columns
experiments.groupby(['catalyst', 'temperature'])['yield'].mean().unstack()

In [ ]:
# Different aggregations for different columns
experiments.groupby('catalyst').agg({
    'conversion': 'mean',
    'selectivity': 'mean',
    'yield': ['mean', 'std'],
    'experiment_id': 'count'
})

In [ ]:
# Named aggregations (cleaner column names)
summary = experiments.groupby('catalyst').agg(
    n_experiments=('experiment_id', 'count'),
    mean_yield=('yield', 'mean'),
    std_yield=('yield', 'std'),
    max_yield=('yield', 'max')
).round(2)

summary

In [ ]:
# Custom aggregation function
def yield_range(x):
    return x.max() - x.min()

experiments.groupby('catalyst')['yield'].agg(['mean', yield_range])

## Merging DataFrames: Combining Data Sources

Real analysis often requires combining data from multiple sources:
- Experiment results + catalyst properties
- Sensor readings + equipment specifications
- Lab data + literature values

### The Four Types of Merges

| Type | What It Keeps | Use When |
|------|---------------|----------|
| `inner` | Only matching rows | You only want complete records |
| `left` | All rows from left table | Your main table is on the left, reference data on right |
| `right` | All rows from right table | Your main table is on the right |
| `outer` | All rows from both | You want to see what's missing |

### The Key Concept: Join Keys

The `on` parameter specifies which column(s) to match. Make sure:
- The columns have the same meaning in both tables
- Values are comparable (watch out for "Pt" vs "Pt/Al2O3")
- Missing keys are handled appropriately (check for NaN after merge)

In [ ]:
# Create catalyst properties table
catalyst_info = pd.DataFrame({
    'catalyst': ['Pt/Al2O3', 'Pd/Al2O3', 'Ru/Al2O3', 'Rh/Al2O3'],
    'metal_loading': [1.0, 0.5, 2.0, 0.3],  # wt%
    'surface_area': [250, 280, 220, 300],  # m2/g
    'cost_per_kg': [50000, 30000, 8000, 80000]  # USD
})

catalyst_info

In [ ]:
# Merge experiments with catalyst info
merged = experiments.merge(catalyst_info, on='catalyst', how='left')
merged.head()

In [ ]:
# Different merge types
# how='left'   - Keep all rows from left DataFrame
# how='right'  - Keep all rows from right DataFrame
# how='inner'  - Keep only matching rows (intersection)
# how='outer'  - Keep all rows from both (union)

# Example: inner merge (Rh/Al2O3 exists in catalyst_info but not in experiments)
inner_merge = experiments.merge(catalyst_info, on='catalyst', how='inner')
print(f"Left: {len(experiments)}, Right: {len(catalyst_info)}, Inner: {len(inner_merge)}")

## Reshaping Data: Pivot and Melt

Data comes in different "shapes," and sometimes you need to reshape it:

### Long vs Wide Format

**Long format** (tidy data):
- Each row is one observation
- Good for analysis and plotting
- Example: Each row = (catalyst, temperature, yield)

**Wide format** (spreadsheet-style):
- Columns represent different conditions
- Easier for humans to read
- Example: Rows = catalysts, Columns = temperatures, Values = yields

### When to Use Which

| Format | Use For |
|--------|---------|
| Long | Most Pandas operations, plotting, ML |
| Wide | Display tables, Excel export, certain visualizations |

### The Transformation Tools

- **`pivot_table()`**: Long → Wide (aggregate if needed)
- **`melt()`**: Wide → Long (unpivot)

In [ ]:
# Pivot: long to wide format
# Create summary table: catalysts as rows, temperatures as columns
pivot_table = experiments.pivot_table(
    values='yield',
    index='catalyst',
    columns='temperature',
    aggfunc='mean'
).round(1)

pivot_table

In [ ]:
# Multiple values in pivot
pivot_multi = experiments.pivot_table(
    values=['conversion', 'selectivity'],
    index='catalyst',
    columns='temperature',
    aggfunc='mean'
).round(2)

pivot_multi

In [ ]:
# Melt: wide to long format
# Useful when data is in "spreadsheet" format
wide_data = pd.DataFrame({
    'catalyst': ['Pt', 'Pd', 'Ru'],
    'yield_300K': [45, 42, 38],
    'yield_400K': [65, 62, 55],
    'yield_500K': [78, 75, 68]
})

print("Wide format:")
print(wide_data)

# Convert to long format
long_data = wide_data.melt(
    id_vars='catalyst',
    var_name='condition',
    value_name='yield'
)

print("\nLong format:")
print(long_data)

## Apply and Transform

In [ ]:
# Apply a function to each row
def categorize_yield(row):
    if row['yield'] >= 70:
        return 'high'
    elif row['yield'] >= 50:
        return 'medium'
    else:
        return 'low'

experiments['yield_category'] = experiments.apply(categorize_yield, axis=1)
experiments[['experiment_id', 'yield', 'yield_category']].head(10)

In [ ]:
# Transform: apply function but keep original shape
# Useful for normalization within groups

# Z-score normalization of yield within each catalyst group
experiments['yield_zscore'] = experiments.groupby('catalyst')['yield'].transform(
    lambda x: (x - x.mean()) / x.std()
)

experiments[['experiment_id', 'catalyst', 'yield', 'yield_zscore']].head(10)

## Working with Dates

In [ ]:
# Extract date components
experiments['year'] = experiments['date'].dt.year
experiments['month'] = experiments['date'].dt.month
experiments['week'] = experiments['date'].dt.isocalendar().week
experiments['day_of_week'] = experiments['date'].dt.day_name()

experiments[['date', 'year', 'month', 'week', 'day_of_week']].head()

In [ ]:
# Group by time period
weekly_avg = experiments.groupby('week')['yield'].mean()

plt.figure(figsize=(10, 4))
weekly_avg.plot(kind='line', marker='o')
plt.xlabel('Week')
plt.ylabel('Average Yield (%)')
plt.title('Weekly Average Yield')
plt.grid(True)
plt.show()

In [ ]:
# Resample time series data
experiments_ts = experiments.set_index('date')

# Weekly average
weekly = experiments_ts['yield'].resample('W').mean()
print(weekly)

## Method Chaining

Combine operations elegantly.

In [ ]:
# Instead of step-by-step:
# df1 = experiments[experiments['temperature'] >= 400]
# df2 = df1.groupby('catalyst')['yield'].mean()
# df3 = df2.sort_values(ascending=False)

# Use method chaining:
result = (
    experiments
    .query('temperature >= 400')  # Filter
    .groupby('catalyst')['yield']  # Group
    .mean()  # Aggregate
    .sort_values(ascending=False)  # Sort
    .round(1)  # Format
)

result

In [ ]:
# Complex analysis in one chain
analysis = (
    experiments
    .assign(yield_category=lambda df: pd.cut(df['yield'], 
                                              bins=[0, 50, 70, 100],
                                              labels=['low', 'medium', 'high']))
    .groupby(['catalyst', 'yield_category'])
    .size()
    .unstack(fill_value=0)
)

analysis

## Summary: The Data Wrangling Toolkit

Intermediate Pandas is about transforming data to answer complex questions. Here's your toolkit:

### Core Operations

| Operation | When to Use | Key Method |
|-----------|-------------|------------|
| **GroupBy** | Compare across categories | `df.groupby('col').agg()` |
| **Merge** | Combine data sources | `df1.merge(df2, on='col')` |
| **Pivot** | Long → Wide format | `df.pivot_table()` |
| **Melt** | Wide → Long format | `df.melt()` |
| **Transform** | Apply function, keep shape | `df.groupby().transform()` |

### The Method Chaining Philosophy

Instead of:
```python
df1 = df[df['temp'] > 400]
df2 = df1.groupby('catalyst')['yield'].mean()
df3 = df2.sort_values(ascending=False)
```

Write:
```python
(df.query('temp > 400')
   .groupby('catalyst')['yield']
   .mean()
   .sort_values(ascending=False))
```

Benefits: More readable, fewer intermediate variables, easier to modify.

### Key Decisions

| Situation | Choice |
|-----------|--------|
| Need to compare groups? | Use `groupby()` |
| Need to combine tables? | Use `merge()` with appropriate `how=` |
| Need to aggregate duplicates? | Use `pivot_table(aggfunc=...)` |
| Need to normalize within groups? | Use `groupby().transform()` |

### Common Gotchas

- `groupby()` returns a GroupBy object, not a DataFrame—add `.mean()`, `.agg()`, etc.
- `merge()` may create more rows than expected if keys aren't unique
- `pivot_table()` aggregates by default (use `aggfunc` to control how)
- Watch for column names becoming tuples after complex aggregations

## Next Steps

Now that you can manipulate data effectively, we'll move to dimensionality reduction—techniques for exploring and visualizing high-dimensional datasets before building models.